In [1]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket
import select
import requests
import threading
import os
import json

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HTTPServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, request_handler):
        self._thread = None
        self.is_stopped = False
        super().__init__(addr, request_handler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        # I attempted to speed up response time by overriding the default 
        # serve_forever with a shorter interval, but it doesn't seem to have worked.
        # oh well. 
        target = super().serve_forever
        self._thread = threading.Thread(target=target, args=(interval,), daemon=True)
        self._thread.start()
        
    def stop(self):
        if self._thread is not None:
            self.shutdown()
            self._thread.join()
            self._thread = None
            self.socket.close()
            self.is_stopped = True
    

In [6]:
def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

def parse_qs(qs):
    return parse_params(urllib.parse.parse_qs(qs))

class HelloProxyError(Exception):
    def __init__(self, code, msg):
        self.response = ProxyResponse(code, msg, {'Connection': 'Close'})
        super().__init__(msg)
        
def get_file_content(fp):
    with open(fp, 'rb') as f:
        return f.read()
        
def content_type_for_file(file):
    ext = os.path.splitext(file)[1]
    if ext == '.json':
        return 'Application/json'
    elif ext == '.css':
        return 'text/css'
    elif ext == '.png':
        return 'image/png'
    elif ext == '.js':
        return 'application/x-javascript'
    elif ext == '.gif':
        return 'image/gif'
    elif ext == '.ico':
        return 'image/icon'
    elif ext == '.html':
        return 'text/html'
    elif ext == '.woff':
        return 'font/woff'
    elif ext == '.ttf':
        return 'application/octet-stream'
    else:
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unrecognized MIME type")
        
def byte_encode(msg):
    if isinstance(msg, str):
        return msg.encode('ascii')
    elif isinstance(msg, bytes):
        return msg
    else:
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Error encoding response data")
        
class ProxyResponse():
    def __init__(self, code, msg, headers=None):
        self.code = code
        self.msg = byte_encode(msg)
        self.headers = headers or {}
        
def proxy_response_for_file(fullpath):
    msg = get_file_content(fullpath)
    
    headers = {
        'Content-Type': content_type_for_file(fullpath),
        'Content-Length': str(len(msg)),
        'Connection': 'Keep-Alive'
    }
    
    return ProxyResponse(200, msg, headers)

def basic_headers_for_json(j):
    return {
        'Content-Type': 'text/json',
        'Content-Length': str(len(j)),
        'Connection': 'Keep-Alive'
    }
    
class HelloProxyHandler(http.server.SimpleHTTPRequestHandler):
    
#     def log_message(self, format, *args):
#         pass
    
    def handle(self):
        self.close_connection = False
        while not self.close_connection:
            r, w, l = select.select([self.request], [], [], 0.5)
            if self.server.is_stopped:
                break
            if r:
                self.handle_one_request()
            else:
                break
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        kw = parse_qs(res.query)
        call = kw.get('call', None)
        try:
            rsp = self._handle_get_internal(res, kw)
        except HelloProxyError as e:
            rsp = e.response
        except Exception as e:
            msg = "???" + str(e.args[0])
            headers = {
                'Connection': 'Close',
                'Content-Type': 'text/plain',
                'Content-Length': str(len(msg))
            }
            rsp = ProxyResponse(http.HTTPStatus.INTERNAL_SERVER_ERROR, msg, headers)
                  
        if rsp.headers['Connection'].lower() == 'close':
            self.close_connection = True
            
        self.send_response(rsp.code)
        for k, v in rsp.headers.items():
            self.send_header(k, v)
        self.end_headers()
        try:
            self.wfile.write(rsp.msg)
            self.wfile.flush()
        except ConnectionAbortedError:
            self.close_connection = True
        
    def _handle_get_internal(self, res, kw):

        # reject paths that try to navigate out of 
        # the local directory
        if ".." in res.path:
            raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Path requests cannot contain access local paths via '..'.")
        
        if res.path.startswith('/webservice'):
            _, ws, handler = res.path.split('/',2)
            if handler[-1] == '/': 
                handler = handler[:-1]
            if handler in ('interface', 'getReport', 'getfile'):
                return self._handle_server_call(kw)
            else:
                raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unsupported service path: '%s'"%handler)
        fullpath = os.path.join(self.server.basepath, res.path.lstrip("\\/"))
        if os.path.exists(fullpath):
            return proxy_response_for_file(fullpath)
        else:
            raise HelloProxyError(http.HTTPStatus.NOT_FOUND, "Failed to find file '%s'"%res.path)
        
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Unreachable code path")
        
    def _handle_fwd_server_call(self, kw):
        url = self.server.proxy_url + self.path
        hfwd = dict(self.headers)
        if self.server.cookies:
            hfwd['Cookie'] = ";".join("%s=%s"%(name, value) for name, value in self.server.cookies.items())
       #print(url)
        fwd = self.server.session.get(url, headers=hfwd, verify=False)
        if fwd.cookies:
            self.server.cookies.update(fwd.cookies)

        msg = fwd.content
        headers = fwd.headers
        headers['Content-Length'] = str(len(msg))
        headers.pop('Transfer-Encoding', None)

        return fwd.status_code, msg, headers
        
        
    USE_DEFAULT = object()
    def _handle_server_call(self, kw):
        
        call = kw.get('call', '').lower()
        func = self.server.get_intercept_handler(call)
        if func:
            try:
                msg = func(kw)
                if not isinstance(msg, str):
                    msg = json.dumps(msg)
                status_code = 200
            except Exception as e:
                status_code = 500
                msg = {"error": str(e)}
                msg = json.dumps(msg)
            headers = basic_headers_for_json(msg)
        else:
            
            # Chrome and requests both don't handle Set-Cookie
            # properly when dealing with localhost (or something like
            # that, whatever something isnt working) so I add 
            # manually based on global server session.
            
            # This has 2 implications: 
            # 1) The client cannot identify themselves across multiple requests
            # 2) The server can only be used for a single client due to needing to store
            #    global data.
            status_code, msg, headers = self._handle_fwd_server_call(kw)
            
        return ProxyResponse(status_code, msg, headers) 
            
    def _server_call_bad_params(self, call, **kw):
        if kw:
            kws = ", ".join("%s=%s" for i in kw.items())
            kws = ": '%s'" % kws
        else:
            kws = ""
        if call is None:
            call = "No call argument provided"
        else:
            call = '%s'%call
            
        msg = "Bad params for server call: %s%s" % (call, kws)
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, msg)
        
class HelloProxyServer(HTTPServer):
    def __init__(self, localaddr, remoteaddr, basepath, https=True):
        self.remoteaddr = remoteaddr
        self.basepath = basepath
        if https:
            scheme = "https"
        else:
            scheme = "http"
        self.proxy_url = "%s://%s:%s"%(scheme, remoteaddr[0], remoteaddr[1])
        self.cookies = requests.cookies.RequestsCookieJar()
        self.session = requests.Session()
        self._thread = None
        self.intercepts = {}
        super().__init__(localaddr, HelloProxyHandler)
        
    def get_intercept_handler(self, call):
        return self.intercepts.get(call, None)
    
    def register_intercept_handler(self, call, func):
        self.intercepts[call] = func
        
    def remove_intercept_handler(self, call):
        del self.intercepts[call]
        
    def __del__(self):
        self.socket.close()

In [11]:
try:
    serv
except NameError:  # first run
    pass
else:
    serv.stop()
    del serv

In [12]:
import clipboard
port = 12348
path = "C:\\Users\\nathan\\Documents\\personal\\test\\uidev_311_dualsens"
serv = HelloProxyServer(('localhost', port), ('localhost', 8001), path, False)
#serv.register_intercept_handler("getstatic", get_gs)
clipboard.copy("http://localhost:%d/hello.html"%port)
serv.serve_forever()

127.0.0.1 - - [28/Feb/2020 14:07:17] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:07:26] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:24:20] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:25:17] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:25:31] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:33:27] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:34:02] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:34:48] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:35:29] "GET /webservice/interface?json=true&call=getStatic HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 14:35:51] "GET /webservice/i

In [ ]:
import types
HelloPro

In [11]:
def gs_311(kw):
    gs = {
           "network":{
              "local":True
           },
           "versions":{
              "rio":"V3.1.150",
              "server":"V3.1.118",
              "type":"Mag",
              "size":15,
              "database":"V2.4",
              "serialNumber":"01CD188F"
           },
           "agitation":{
              "pvUnit":"RPM",
              "manUnit":"%",
              "manName":"Percent Power",
              "pvDecimals":0,
              "manDecimals":1
           },
           "temperature":{
              "pvUnit":"°C",
              "manUnit":"%",
              "manName":"Main Heater Duty",
              "pvDecimals":1,
              "manDecimals":1,
              "inputs":2
           },
           "do":{
              "pvUnit":"%",
              "manUpUnit":"%",
              "manDownUnit":"%",
              "manUpName":"O2",
              "manDownName":"N2",
              "pvDecimals":1,
              "manUpDecimals":1,
              "manDownDecimals":1,
              "inputs":2
           },
           "ph":{
              "pvUnit":"",
              "manUpUnit":"%",
              "manDownUnit":"%",
              "manUpName":"Base",
              "manDownName":"CO2",
              "pvDecimals":2,
              "manUpDecimals":1,
              "manDownDecimals":1,
              "inputs":2,
              "pHAUseTempComp":1,
              "pHBUseTempComp":1
           },
           "pressure":{
              "pvUnit":"psi",
              "pvDecimals":1
           },
           "level":{
              "pvUnit":"L",
              "pvDecimals":1,
              "enableLevel":1
           },
           "condenser":{
              "pvUnit":"°C",
              "manName":"Filter Oven Heater Duty",
              "manUnit":"%",
              "pvDecimals":1,
              "manDecimals":1
           },
           "maingas":{
              "pvUnit":"L/min",
              "manName":"Total Flow",
              "manUnit":"L/min",
              "pvDecimals":2,
              "manDecimals":2
           },
           "mfcs":{
              "airPvDecimals":2,
              "co2PvDecimals":2,
              "n2PvDecimals":2,
              "o2PvDecimals":2
           }
        }
    
    gs['ph']['inputs'] = 2
    gs['do']['inputs'] = 2
    gs['temperature']['inputs'] = 2
    gs['ph']['pHAUseTempComp'] = 1
    gs['ph']['pHBUseTempComp'] = 0
    
    return json.dumps(gs)

serv.register_intercept_handler("getstatic", gs_311)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 64690)
----------------------------------------


Traceback (most recent call last):
  File "c:\program files\python35\lib\socketserver.py", line 625, in process_request_thread
    self.finish_request(request, client_address)
  File "c:\program files\python35\lib\socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\program files\python35\lib\socketserver.py", line 681, in __init__
    self.handle()
  File "<ipython-input-8-dd23b4974dbb>", line 88, in handle
    self.handle_one_request()
  File "c:\program files\python35\lib\http\server.py", line 390, in handle_one_request
    self.raw_requestline = self.rfile.readline(65537)
  File "c:\program files\python35\lib\socket.py", line 575, in readinto
    return self._sock.recv_into(b)
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


In [34]:
gbc = """<?xml version="1.0" encoding="windows-1252" standalone="no" ?>
<Reply>
<Result>True</Result>
<Message><Cluster>
<Name></Name>
<NumElts>4</NumElts>
<Cluster>
<Name>CalpHA</Name>
<NumElts>3</NumElts>
<DBL>
<Name>Slope</Name>
<Val>1231293123.87480187416077</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>1.25340652465820</Val>
</DBL>
<DBL>
<Name>Temp(C)</Name>
<Val>25.00000000000000</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalDOA</Name>
<NumElts>2</NumElts>
<DBL>
<Name>Slope</Name>
<Val>47.77952194213867</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-95.45653533935547</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalpHB</Name>
<NumElts>3</NumElts>
<DBL>
<Name>Slope</Name>
<Val>4.12682008743286</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-4.25458002090454</Val>
</DBL>
<DBL>
<Name>Temp(C)</Name>
<Val>0.00000000000000</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalDOB</Name>
<NumElts>2</NumElts>
<DBL>
<Name>Slope</Name>
<Val>73.77993011474609</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-64.69270324707031</Val>
</DBL>
</Cluster>
</Cluster>
</Message>
</Reply>"""

def getbagconfig(kw):
    return gbc

serv.register_intercept_handler("getbagconfig", getbagconfig)

In [20]:
lead = """*Configure the bioreactor's system variables:

Level -> Enable Sensor = 0.

Reload the Hello UI. 
*Navigate to the Calibration menu and note whether the Level menu button is present.
+Level button not present. 
*Configure the bioreactor's system variables:

Level -> Enable Sensor = 1."""

text = """#Subtest #%(subtest)d
*Configure the bioreactor's bioreactor configuration.json file to have the following values:

Sensors.Temp Hardware = %(temp_hardware)s
Sensors.DO Hardware = %(do_hardware)s
Sensors.pH Hardware = %(ph_hardware)s

Reboot the RIO and reload the Hello UI when finished.
*Log into the Hello UI and navigate to Actions. Note whether "Select Sensors" is present.
+Button %(ss_present)s present.%(maybe_ss_sliders)s
*Navigate to the calibration tab and note the pH and DO buttons.
+pH A: %(pha)s, pH B: %(phb)s, DO A: %(doa)s, DO B: %(dob)s
"""

def pres(v):
    return "visible" if v == 1 else "hidden"

def ab(v):
    a = 0
    b = 0
    if v == 0:
        a = 1
    if v == 1:
        b = 1
    if v == 2:
        a = 1
        b = 1
    return pres(a), pres(b)
    

data = {
    "temp_hardware": 0,
    "do_hardware": 0,
    "ph_hardware": 0,
    "ss_present": 0,
    "pha": 0,
    "phb": 0,
    "doa": 0,
    "dob": 0,
    "maybe_ss_sliders": 0,
    "subtest": 0
}

ss_sliders = {
    "ph": 0,
    "do": 0,
    "temp": 0
}

sliders_text = """
*Enter the Select Sensor Menu and note the visible sliders
+pH: %(ph)s, DO: %(do)s, Temp: %(temp)s"""

def _sl(v):
    return "visible" if v == 2 else "hidden"

def sliders(t, d, p):
    ss_sliders["ph"] = _sl(p)
    ss_sliders["temp"] = _sl(t)
    ss_sliders["do"] = _sl(d)
import io,sys, clipboard
buf = io.StringIO()
_stdout = sys.stdout
sys.stdout = buf
print(lead)
i = 0
for t in range(3):
    data['temp_hardware'] = t
    for d in range(3):
        data['do_hardware'] = d
        for p in range(3):
            data['ph_hardware'] = p
            
            i += 1
            data['subtest'] = i
            
            if t == 2 or d == 2 or p == 2:
                data['ss_present'] = "is"
                sliders(t, d, p)
                data["maybe_ss_sliders"] = sliders_text % ss_sliders
                
            else:
                data['ss_present'] = "is not"
                data['maybe_ss_sliders'] = ""
                
            a,b = ab(p)
            data['pha'] = a
            data['phb'] = b
            
            a,b = ab(d)
            data['doa'] = a
            data['dob'] = b
            
            print(text%data)
sys.stdout = _stdout
v = buf.getvalue().strip()
print(v)
clipboard.copy(v)


*Configure the bioreactor's system variables:

Level -> Enable Sensor = 0.

Reload the Hello UI. 
*Navigate to the Calibration menu and note whether the Level menu button is present.
+Level button not present. 
*Configure the bioreactor's system variables:

Level -> Enable Sensor = 1.
#Subtest #1
*Configure the bioreactor's bioreactor configuration.json file to have the following values:

Sensors.Temp Hardware = 0
Sensors.DO Hardware = 0
Sensors.pH Hardware = 0

Reboot the RIO and reload the Hello UI when finished.
*Log into the Hello UI and navigate to Actions. Note whether "Select Sensors" is present.
+Button is not present.
*Navigate to the calibration tab and note the pH and DO buttons.
+pH A: visible, pH B: hidden, DO A: visible, DO B: hidden

#Subtest #2
*Configure the bioreactor's bioreactor configuration.json file to have the following values:

Sensors.Temp Hardware = 0
Sensors.DO Hardware = 0
Sensors.pH Hardware = 1

Reboot the RIO and reload the Hello UI when finished.
*Log i